# Make Granule List

In [1]:
# import utility function for making granule list
from utils import make_granule_list

In [2]:
# # function to convert shapefile to geojson - if needed
# shp2geojson_nsidc('shapefiles/jakobshavn_small.shp')

In [3]:
# set parameters and make the list
geojson = 'jakobshavn_small.geojson'
start_date = '2021-05-01'
end_date = '2021-09-15'
outname = 'granule_lists/jakobshavn_small_2021.csv'

make_granule_list(geojson, start_date, end_date, outname)

Found 36 ATL03 version 005 granules over jakobshavn_small.geojson between 2021-05-01 and 2021-09-15.
Wrote file: granule_lists/jakobshavn_small_2021.csv


In [8]:
# make sure that it worked by reading the file into a DataFrame and displaying it
import pandas as pd
pd.read_csv(outname,header=None)

,0,1
0,ATL03_20210503103512_06051105_005_01.h5,geojsons/jakobshavn_small.geojson
1,ATL03_20210506214200_06581103_005_01.h5,geojsons/jakobshavn_small.geojson
2,ATL03_20210507102653_06661105_005_01.h5,geojsons/jakobshavn_small.geojson
3,ATL03_20210510213341_07191103_005_01.h5,geojsons/jakobshavn_small.geojson
4,ATL03_20210511101834_07271105_005_01.h5,geojsons/jakobshavn_small.geojson
5,ATL03_20210514212522_07801103_005_01.h5,geojsons/jakobshavn_small.geojson
6,ATL03_20210518211701_08411103_005_01.h5,geojsons/jakobshavn_small.geojson
7,ATL03_20210601091116_10471105_005_01.h5,geojsons/jakobshavn_small.geojson
8,ATL03_20210605090256_11081105_005_01.h5,geojsons/jakobshavn_small.geojson
9,ATL03_20210608200944_11611103_005_01.h5,geojsons/jakobshavn_small.geojson
